In [31]:
# !pip install sounddevice
# !pip install pyaudio
!pip install pyaudio


In [23]:
import os
import librosa
import numpy as np
from keras.models import load_model
import pandas as pd
import speech_recognition as sr
from nltk.sentiment import SentimentIntensityAnalyzer

caminho_do_audio = "audios teste reconhecimento/Audio 1 - reconhecimento.wav"

# Função para transcrever áudio
def transcrever_audio(caminho_audio):
    recognizer = sr.Recognizer()

    with sr.AudioFile(caminho_audio) as audio_file:
        audio = recognizer.record(audio_file)

        try:
            texto_transcrito = recognizer.recognize_google(audio, language="pt-BR")  # language="en" para inglês, "pt-BR" para português
            return texto_transcrito
        except sr.UnknownValueError:
            print("Não foi possível reconhecer o áudio")
        except sr.RequestError as e:
            print(f"Erro na requisição para o serviço de reconhecimento de fala: {e}")

# Função para analisar sentimento
def analisar_sentimento(texto):
    sia = SentimentIntensityAnalyzer()
    resultado = sia.polarity_scores(texto)
    resultado = resultado['compound']
    return resultado

#     if resultado['compound'] >= 0.05:
#         return 'Positivo'
#     elif resultado['compound'] <= -0.05:
#         return 'Negativo'
#     else:
#         return 'Neutro'

# Função para extrair características do áudio
def extract_features(caminho_audio):
    y, sr = librosa.load(caminho_audio)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
    mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr), axis=1)
    features = np.hstack([mfccs, chroma, mel])
    return features

# # Diretório contendo os arquivos de áudio
# audio_directory = 'audios classificados/'

# # Escolha o arquivo de áudio para análise
# filename = 'nome_do_arquivo.wav'  # Substitua pelo nome do arquivo desejado

# # Construir o caminho completo do arquivo
# file_path = os.path.join(audio_directory, filename)

# Carregar o modelo treinado
model = load_model("seu_modelo.h5")

# Inicializar DataFrame para armazenar resultados
result_df = pd.DataFrame(columns=["Arquivo", "Resultado"])

# try:
# Extrair características do áudio
audio_features = extract_features(caminho_do_audio)

# Ajustar as características para corresponder à entrada do modelo
audio_features_input = np.array(audio_features).reshape(1, -1)

# Obter o score de sentimento usando o modelo treinado
sentiment_score = model.predict(audio_features_input)

# Transcrever áudio
texto_transcrito = transcrever_audio(caminho_do_audio)

# Analisar sentimento do texto transcrito
sentimento_texto = analisar_sentimento(texto_transcrito)

# Normalizar resultado['compound'] para o intervalo [0, 1]
normalized_compound = (sentimento_texto + 1) / 2

# Calcular média entre normalized_compound e sentiment_score[0][0]
media_resultado = (normalized_compound + sentiment_score[0][0]) / 2

# # Adicionar resultado ao DataFrame
# result_df = result_df.append({"Arquivo": filename, "Resultado": media_resultado}, ignore_index=True)

# Exibir o resultado
print(texto_transcrito)
print(media_resultado)

# except Exception as e:
#     print(f"Erro ao processar o arquivo {filename}: {e}")


1/1 [==============================] - 0s 69ms/step
o rato roeu a roupa do rei de Roma
0.2500801224960014


In [37]:
#GRAVAÇÃO EM TEMPO REAL

import os
import librosa
import numpy as np
from keras.models import load_model
import pandas as pd
import speech_recognition as sr
import pyaudio
import wave
from nltk.sentiment import SentimentIntensityAnalyzer

# Função para gravar áudio em tempo real
def gravar_audio():
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "gravacao_temp.wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Gravando...")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Gravação concluída.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(pyaudio.PyAudio().get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    return WAVE_OUTPUT_FILENAME

# Função para transcrever áudio
def transcrever_audio(caminho_audio):
    recognizer = sr.Recognizer()

    with sr.AudioFile(caminho_audio) as audio_file:
        audio = recognizer.record(audio_file)

        try:
            texto_transcrito = recognizer.recognize_google(audio, language="pt-BR")
            return texto_transcrito
        except sr.UnknownValueError:
            print("Não foi possível reconhecer o áudio")
        except sr.RequestError as e:
            print(f"Erro na requisição para o serviço de reconhecimento de fala: {e}")

# Função para analisar sentimento
def analisar_sentimento(texto):
    sia = SentimentIntensityAnalyzer()
    resultado = sia.polarity_scores(texto)
    resultado = resultado['compound']
    return resultado

# Função para extrair características do áudio
def extract_features(caminho_audio):
    y, sr = librosa.load(caminho_audio)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
    mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr), axis=1)
    features = np.hstack([mfccs, chroma, mel])
    return features

# Carregar o modelo treinado
model = load_model("seu_modelo.h5")

# Gravar áudio em tempo real
caminho_do_audio = gravar_audio()

# Transcrever áudio
texto_transcrito = transcrever_audio(caminho_do_audio)

# Analisar sentimento do texto transcrito
sentimento_texto = analisar_sentimento(texto_transcrito)

# Extrair características do áudio
audio_features = extract_features(caminho_do_audio)

# Ajustar as características para corresponder à entrada do modelo
audio_features_input = np.array(audio_features).reshape(1, -1)

# Obter o score de sentimento usando o modelo treinado
sentiment_score = model.predict(audio_features_input)

# Normalizar resultado['compound'] para o intervalo [0, 1]
normalized_compound = (sentimento_texto + 1) / 2

# Calcular média entre normalized_compound e sentiment_score[0][0]
media_resultado = (normalized_compound + sentiment_score[0][0]) / 2

# Exibir o resultado
print("Texto transcrito:", texto_transcrito)
print("Resultado médio:", media_resultado)


Gravando...
Gravação concluída.
1/1 [==============================] - 0s 65ms/step
Texto transcrito: e depois esqueci de falar contigo aquele SMS que
Resultado médio: 0.25018084778275806


In [52]:
#GRAVAÇÃO EM TEMPO REAL V2

import os
import librosa
import numpy as np
from keras.models import load_model
import pandas as pd
import speech_recognition as sr
import pyaudio
import wave
from nltk.sentiment import SentimentIntensityAnalyzer
from tkinter import *
from threading import Thread

class GravadorAudio:
    def __init__(self, root):
        self.root = root
        self.root.title("Gravador de Áudio")
        
        self.gravando = False
        self.caminho_do_audio = None

        # Configurar botões
        self.iniciar_botao = Button(root, text="Iniciar Gravação", command=self.iniciar_gravacao)
        self.iniciar_botao.pack(pady=10)

        self.parar_botao = Button(root, text="Parar Gravação", command=self.parar_gravacao, state=DISABLED)
        self.parar_botao.pack(pady=10)

        self.resultado_label = Label(root, text="")
        self.resultado_label.pack(pady=10)

        # Carregar o modelo treinado
        self.model = load_model("seu_modelo.h5")

    def iniciar_gravacao(self):
        self.gravando = True
        self.iniciar_botao.config(state=DISABLED)
        self.parar_botao.config(state=NORMAL)

        # Iniciar thread para gravar áudio
        Thread(target=self.gravar_audio).start()

    def parar_gravacao(self):
        self.gravando = False
        self.iniciar_botao.config(state=NORMAL)
        self.parar_botao.config(state=DISABLED)

        # Transcrever áudio e realizar análises
        texto_transcrito = self.transcrever_audio(self.caminho_do_audio)
        sentimento_texto = self.analisar_sentimento(texto_transcrito)
        audio_features = self.extract_features(self.caminho_do_audio)
        audio_features_input = np.array(audio_features).reshape(1, -1)
        sentiment_score = self.model.predict(audio_features_input)
        normalized_compound = (sentimento_texto + 1) / 2
        media_resultado = (normalized_compound + sentiment_score[0][0]) / 2

        # Exibir resultados
        self.resultado_label.config(text=f"Texto transcrito: {texto_transcrito}\nResultado médio: {media_resultado}")

    def gravar_audio(self):
        CHUNK = 1024
        FORMAT = pyaudio.paInt16
        CHANNELS = 1
        RATE = 44100
        RECORD_SECONDS = 5
        WAVE_OUTPUT_FILENAME = "gravacao_temp.wav"

        p = pyaudio.PyAudio()

        stream = p.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        frames_per_buffer=CHUNK)

        frames = []

        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            if not self.gravando:
                break
                

# Iniciar loop principal da janela
root.mainloop()
